## Identify the mode of the given images 

In [14]:
image= Image.open("./images/disc.tif")
mode = image.mode
print(mode)

I;16B


## Code to generate Random Images  

In [23]:
%%writefile test2.py

#python3 gen_images.py --input input_images/ --out-dims 1024 1024 --nout 1000 --output output_images/


import os
import random
import argparse
from PIL import Image

def generate_images(input_folder, M, output_folder, nout):
    # Checking if the input folder exists
    if not os.path.isdir(input_folder):
        print(f"Input folder {input_folder} does not exist.")
        return

    # Creating output folder
    os.makedirs(output_folder, exist_ok=True)

    # Reading 4 images from the input folder
    input_images = [Image.open(os.path.join(input_folder, file)).convert('I') for file in os.listdir(input_folder) if file.endswith('.tif')]
    N = len(input_images)

    # Computing k based on the input image sizes and M
    k = M // max([max(img.size) for img in input_images])

    for i in range(nout):
        # Generating M x M pixel output images in I mode
        output_image = Image.new('I', (M, M))

        # List to store the positions and sizes of all images that have been placed
        placed_images = []

        for j in range(N):
            for _ in range(k):
                # Scaling each shape randomly by a factor of 1 to 0.75
                scale_factor = random.uniform(0.75, 1)
                img = input_images[j].resize((int(input_images[j].width * scale_factor), int(input_images[j].height * scale_factor)))

                # Rotating each shape randomly by 0 to 90 degrees
                img = img.rotate(random.uniform(0, 90))

                #Placing the shapes at random position without getting cut off at the boundary or overlap other shapes
                for _ in range(100):  # Try up to 100 times
                    max_x = M - img.width
                    max_y = M - img.height
                    pos_x = random.randint(0, max_x)
                    pos_y = random.randint(0, max_y)

                    # Checking if this position overlaps with any existing shapes
                    if any((pos_x < x + w and pos_x + img.width > x and pos_y < y + h and pos_y + img.height > y) for x, y, w, h in placed_images):
                        continue  # This position overlaps

                    # This position does not overlap, placing the shape here
                    output_image.paste(img, (pos_x, pos_y))
                    placed_images.append((pos_x, pos_y, img.width, img.height))
                    break
                else:
                    print(f"Could not place image {j} after 100 attempts. Skipping this image.")

        # Save the output image
        output_image.save(os.path.join(output_folder, f'output_{i}.png'))

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Generate images.')
    parser.add_argument('--input', type=str, required=True, help='Input folder')
    parser.add_argument('--out-dims', type=int, nargs=2, required=True, help='Output image dimensions')
    parser.add_argument('--nout', type=int, required=True, help='Number of output images')
    parser.add_argument('--output', type=str, default='./output/', help='Output folder')
    args = parser.parse_args()

    generate_images(args.input, args.out_dims[0], args.output, args.nout)


                   


Writing test2.py


## DeepLearning Code to Identify the object in the generated Images

In [1]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5")) # Downloading the YOLO model 
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "output_0.png"), output_image_path=os.path.join(execution_path , "image_new.png")) # Replace 'image.jpg' with your image path

for eachObject in detections:
    print(eachObject["name"] , " : " , eachObject["percentage_probability"], " : ", eachObject["box_points"] )


2024-03-03 20:53:24.878104: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:53:24.922677: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:53:24.923401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 20:53:25.668056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1/1 [==============================] - 1s 1s/step


In [1]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(os.path.join(execution_path, "yolo.h5"))
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path, "output_0.png"), output_image_path=os.path.join(execution_path, "image_new.png"))

#Detections in a variable
detected_objects = []

for eachObject in detections:
    object_info = {
        "name": eachObject["name"],
        "percentage_probability": eachObject["percentage_probability"],
        "box_points": eachObject["box_points"]
    }
    detected_objects.append(object_info)

# Manipulating the variable as needed
for object_info in detected_objects:
    print(object_info["name"], " : ", object_info["percentage_probability"], " : ", object_info["box_points"])


2024-03-03 20:56:46.302201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:56:46.347946: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:56:46.348689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 20:56:47.096393: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1/1 [==============================] - 1s 1s/step


In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np  # Import numpy library

#Image size and number of classes
img_size = (256, 256)  
num_classes = 4  

# Loading training data using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize and split

train_generator = train_datagen.flow_from_directory(
    "training_set",
    target_size=img_size,
    batch_size=32,  
    class_mode="categorical",
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    "training_set",
    target_size=img_size,
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# Defining the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=img_size + (3,)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="softmax")
])

# Compiling the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
model.fit(
    train_generator,
    epochs=10,  
    validation_data=validation_generator
)

# Loading and pre-processing the final image
img = tf.keras.preprocessing.image.load_img("gear1.png", target_size=img_size)
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)  
x /= 255.0  

# Predicting the class of the image
predictions = model.predict(x)
predicted_class = np.argmax(predictions[0])
predicted_shape = train_generator.class_indices.get(predicted_class, "unknown")

#Prediction
print(f"Predicted shape: {predicted_shape}")


Found 329 images belonging to 4 classes.
Found 80 images belonging to 4 classes.
Epoch 1/10
11/11 [==============================] - 17s 1s/step - loss: 10.4349 - accuracy: 0.3070 - val_loss: 1.0200 - val_accuracy: 0.5125
Epoch 2/10
11/11 [==============================] - 20s 2s/step - loss: 0.9043 - accuracy: 0.5836 - val_loss: 0.6367 - val_accuracy: 0.7750
Epoch 3/10
11/11 [==============================] - 25s 2s/step - loss: 0.5180 - accuracy: 0.7629 - val_loss: 0.5228 - val_accuracy: 0.7625
Epoch 4/10
11/11 [==============================] - 23s 2s/step - loss: 0.3627 - accuracy: 0.7994 - val_loss: 0.3862 - val_accuracy: 0.7875
Epoch 5/10
11/11 [==============================] - 22s 2s/step - loss: 0.2974 - accuracy: 0.8571 - val_loss: 0.3814 - val_accuracy: 0.8250
Epoch 6/10
11/11 [==============================] - 21s 2s/step - loss: 0.2469 - accuracy: 0.8815 - val_loss: 0.4885 - val_accuracy: 0.8000
Epoch 7/10
11/11 [==============================] - 22s 2s/step - loss: 0.2461